「TorchTextを使用した言語翻訳」
======
【原題】Language Translation with TorchText

【原著】[Ben Trevett](https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb)、および [Seth Weidman](https://github.com/SethHWeidman/)

【元URL】https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　御手洗 拓真

【日付】2020年11月03日

【チュトーリアル概要】

TorchTextを使用してデータを前処理し、ドイツ語を英語に翻訳するモデルを構築します。

（日本語訳注：2020-11-02時点における、Google Colaboratoryのデフォルトのtorchtextのバージョンでは動作しない箇所があります。

日本語版チュートリアルでは最初にtorchtextの最新バージョンv0.8を指定してインストールするセルを追加しています。

また、この後の工程で必要となるため、spacyの最新バージョンもv2.3.2もインストールしています。

インストールを実行した後は、有効化するために、**ランダイムを再起動してください**。上部メニューの「ランタイム」→「ランタイムの再起動」）


In [1]:
!pip install torchtext==0.8.0 spacy==2.3.2

TorchTextを使った言語翻訳 
=========================

本チュートリアルでは、`torchtext`で用意されている便利なクラス群を使用して、英語の文をドイツ語に翻訳します。

データセットには、英語とドイツ語の文を含む、有名なデータセットを使用します。

そして、データに対して前処理を行い、ドイツ語の文を英語に翻訳する、「アテンション付きsequence-to-sequentモデル」を構築します。


本チュートリアルはPyTorch コミュニティのメンバーである [Ben Trevett氏](https://github.com/bentrevett)の[チュートリアル ](https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb) をベースにしており、[Seth Weidman](https://github.com/SethHWeidman/)が Ben の許可を得て作成しました。

本チュートリアルを終えると、以下の内容ができるようになります。

-   `torchtext`に実装されている以下の便利なクラスを使用し、文章データを前処理し、NLPモデリングのために一般的に使用される形式に変換
    - [`TranslationDataset`](https://torchtext.readthedocs.io/en/latest/datasets.html#torchtext.datasets.TranslationDataset) 
    - [`Field`](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.Field)
    - [`BucketIterator`](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.BucketIterator)

Field and TranslationDataset
---

`torchtext`には、言語翻訳モデルを作成する際に、イテレータ処理（繰り返し処理）が簡単に可能となるデータセットを作成する、便利なクラスが用意されています。

ここで重要となるクラスは、各テキストの前処理方法を指定する`Field`です。

そしてもう一つの重要なクラスが`TranslationDataset`です。




`torchtext`には様々なデータセットが用意されています。

本チュートリアルでは、 約30,000文（平均13語程度の長さ）の英語とドイツ語でが収録されている[Multi30k dataset](https://github.com/multi30k/dataset)を使用します。



**注意**： 

本チュートリアルで使用するトークン化には[*Spacy*](https://spacy.io)が必要です。

*Spacy*を使用しているのは、英語以外の言語でのトークン化も強力にサポートしているためです。


`torchtext` は `basic_english` トークナイザや英語用の他のトークナイザ（例えば[Moses](https://bitbucket.org/luismsgomes/mosestokenizer/src/default/) など) はサポートしていますが、言語翻訳用に使える複数の言語語用のトークナイザはサポートしていません。

そのため、言語翻訳には Spacy を利用するのが最適です。

-----


本チュートリアルを実行するには、まず`pip`または`condaを`使用して`spacy`をインストールします。

（日本語訳：日本語版では最初に既にspacyをインストールしました。）

<br>

次に、英語とドイツ語のSpacyトークナイザの生データをダウンロードします。

```
python -m spacy download en
python -m spacy download de
```

In [2]:
# 日本語訳注：英語とドイツ語のSpacyトークナイザの生データをダウンロードするセルを追加しています
!python -m spacy download en
!python -m spacy download de

     |████████████████████████████████| 12.1MB 774kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp36-none-any.whl size=12047110 sha256=09003338597b73b9d303468c56976605f3239dbb7007f636e00d6fa82d457296
  Stored in directory: /tmp/pip-ephem-wheel-cache-osxcv__m/wheels/2b/3f/41/f0b92863355c3ba34bb32b37d8a0c662959da0058202094f46
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 529kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.3.0-cp36-none-any.whl size=14907580 sha256=73e194e8c5ae16f26b0b431d8a3cf

Spacyがインストールされている場合、以下のコードを実行すると`Field`で定義されたトークナイザーの設定に基づいて、`TranslationDataset`の各文は、トークン化されます。

In [3]:
%matplotlib inline

In [4]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize = "spacy",
            tokenizer_language="de",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                    fields = (SRC, TRG))

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 647kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 169kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 165kB/s]
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


上記の実行により`train_data`が定義できたので、`torchtext`の`Field` にある、非常に便利な機能（`build_vocab`メソッド）を使用できます。

`build_vocab`メソッドによって、各言語データのvocabulary（単語集）が自動作成できます。

In [5]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

上記のコードを実行後、`SRC.vocab.stoi`と呼び出すことで、以下の辞書形式のデータが取得できます。

- キー：語彙のトークン
- 値：トークンに対応するインデックス

<br>

そして`SRC.vocab.itos`は、同じ内容の値が入ったリストとなります。

本チュートリアルではこの知識をフルに活用することはありませんが、他のNLPタスクではこの知識が役に立つケースもあるかと思います。

In [7]:
# 日本語版注：追記
print(SRC.vocab.stoi)
print("------------")
print(SRC.vocab.itos)
print(SRC.vocab['<sos>'])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f61e36c9cf8>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, '.': 4, 'ein': 5, 'einem': 6, 'in': 7, 'eine': 8, ',': 9, 'und': 10, 'mit': 11, 'auf': 12, 'mann': 13, 'einer': 14, 'der': 15, 'frau': 16, 'die': 17, 'zwei': 18, 'einen': 19, 'im': 20, 'an': 21, 'von': 22, 'sich': 23, 'dem': 24, 'mädchen': 25, 'junge': 26, 'vor': 27, 'zu': 28, 'steht': 29, 'männer': 30, 'sitzt': 31, 'hund': 32, 'den': 33, 'straße': 34, 'während': 35, 'gruppe': 36, 'hält': 37, 'spielt': 38, 'das': 39, 'hemd': 40, 'personen': 41, 'über': 42, 'drei': 43, 'eines': 44, 'frauen': 45, 'blauen': 46, 'neben': 47, 'ist': 48, 'kind': 49, 'roten': 50, 'weißen': 51, 'stehen': 52, 'sitzen': 53, 'menschen': 54, 'am': 55, 'aus': 56, 'spielen': 57, 'durch': 58, 'bei': 59, 'geht': 60, 'trägt': 61, 'fährt': 62, 'wasser': 63, 'um': 64, 'kinder': 65, 'kleines': 66, 'person': 67, 'macht': 68, 'springt': 69, 'kleiner': 70, 'schwarzen': 71, 

`BucketIterator `
-----------------

最後に紹介する、`torchtext `特有の機能は、`BucketIterator`です。

これは第一引数として`TranslationDataset`を取り、簡単に使用できます。


具体的な使い方はドキュメントに書かれている通り、以下の通りです。

[Pytorch公式ドキュメントのBucketIteratorの項目](https://torchtext.readthedocs.io/en/latest/data.html#bucketiterator)



まず、長さがだいたい同じくらいの文をまとめて扱うイテレータを定義します。

このイテレータを使って、エポックごとに、データをシャッフルしてミニバッチを作成します。

この際に、バッチごとに最低限の長さをそろえるのに必要な、パディング操作も実施します。

これらのバケット化の手順の詳細については、[Pytorch公式ドキュメントのpoolの項目](https://torchtext.readthedocs.io/en/latest/data.html#pool)を参照してください。



In [8]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


これらのイテレータは、`DataLoaders`と同様に呼び出すことができます。

したがって`train`関数と`evaluate`関数では以下のようにシンプルな形で呼び出されます。

（日本語訳注：train関数とevaluate関数は、このあと実装します。）

```
for i, batch in enumerate(iterator)
```

各`batch`は `src`と `trg`の属性変数を持っています。

```
src = batch.src 
trg = batch.trg
```

`nn.Module`と`Optimizerの`定義 
---------------------------------

`torchtext`に関しての説明内容は以上となります。

データセットが構築され、イテレータが定義されていれば、本チュートリアルの残る部分は、`Optimizer`の定義と、モデルを`nn.Module`として定義して、訓練を実施するだけです。


本チュートリアルのモデルは、[こちら](https://arxiv.org/abs/1409.0473)に記載されているアーキテクチャに従っています。


(なお、さらに多くの解説が追加された資料は、[こちら](https://github.com/SethHWeidman/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb)で公開されています）

**注意**：

今回使用するモデルは、言語翻訳に使用できるモデルの一例に過ぎません。

このモデルを取り上げたのは、翻訳タスクにおいて標準的だからであり、今回のモデルを実際の翻訳プロジェクトに推奨しているからではありません。

ご存じのように、最先端のモデルは現在、トランスフォーマーをベースとしています。

なお、トランスフォーマー層を実装するためのPyTorchの機能は[こちら](https://pytorch.org/docs/stable/nn.html#transformer-layers)から確認することができます。

また、特に注意して頂きたいのですが、以降のモデルで仕様している「アテンション」は、トランスフォーマーモデルにおけるマルチヘッドのセルフ・アテンションとは異なるものです。
___

In [9]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # デコーダへの最初の入力は<sos>トークンです
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,856,653 trainable parameters


**注意**：

言語翻訳モデルの性能をスコアリングする場合には、単なるパディング部分のインデックスは、損失関数の計算時には無視するよう、NN`.CrossEntropyLoss`関数に設定する必要があります。




In [10]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


最後に、このモデルを訓練して評価します（5分程度かかります）。

In [11]:
import math
import time


def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) # 教師強制をオフにします

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Time: 0m 28s
	Train Loss: 5.687 | Train PPL: 294.976
	 Val. Loss: 5.250 |  Val. PPL: 190.606
Epoch: 02 | Time: 0m 28s
	Train Loss: 5.014 | Train PPL: 150.546
	 Val. Loss: 5.126 |  Val. PPL: 168.299
Epoch: 03 | Time: 0m 28s
	Train Loss: 4.747 | Train PPL: 115.274
	 Val. Loss: 5.015 |  Val. PPL: 150.689
Epoch: 04 | Time: 0m 28s
	Train Loss: 4.600 | Train PPL:  99.474
	 Val. Loss: 4.840 |  Val. PPL: 126.530
Epoch: 05 | Time: 0m 29s
	Train Loss: 4.434 | Train PPL:  84.279
	 Val. Loss: 4.850 |  Val. PPL: 127.731
Epoch: 06 | Time: 0m 28s
	Train Loss: 4.342 | Train PPL:  76.885
	 Val. Loss: 4.725 |  Val. PPL: 112.774
Epoch: 07 | Time: 0m 29s
	Train Loss: 4.229 | Train PPL:  68.658
	 Val. Loss: 4.721 |  Val. PPL: 112.286
Epoch: 08 | Time: 0m 29s
	Train Loss: 4.132 | Train PPL:  62.308
	 Val. Loss: 4.669 |  Val. PPL: 106.609
Epoch: 09 | Time: 0m 29s
	Train Loss: 4.026 | Train PPL:  56.015
	 Val. Loss: 4.580 |  Val. PPL:  97.530
Epoch: 10 | Time: 0m 29s
	Train Loss: 3.957 | Train PPL

ネクスト・ステップ 
------------

-   `torchtext を`使った、Ben Trevett氏によるチュートリアルの続きは、[こちら](https://github.com/bentrevett/)になります
-   他の`torchtext `の機能を使用したチュートリアルや、本チュートリアルシリーズの1つ目「TransformerとTorchTextを用いたsequence-to-sequenceモデルの学習（SEQUENCE-TO-SEQUENCE MODELING WITH NN.TRANSFORMER AND TORCHTEXT）」のnext word predictionタスクで使用した`nn.Transformer `を参考に、工夫を加えてみてください。